# Initialization

In [1]:
from logging import ERROR
import os
from pathlib import Path
import signal
from subprocess import CalledProcessError
import time

# PyBoolNet initialization
from pyboolnet.repository import get_all_names, get_bnet
from pyboolnet.file_exchange import bnet2primes
from pyboolnet.trap_spaces import compute_trap_spaces
from pyboolnet import log
log.setLevel(ERROR)

# mpbn init
import mpbn

# BioLQM init
import biolqm
from py4j.protocol import Py4JJavaError

# trappist init
import trappist

# timeout with signals
class TimeOutException(Exception):
   pass

def handler(_signnum, _frame):
    print("Timeout")
    raise TimeOutException()
    
signal.signal(signal.SIGALRM, handler)

This notebook has been executed using the docker image `colomoto/colomoto-docker` built on `2022-06-08`

<Handlers.SIG_DFL: 0>

In [2]:
# compute trapspaces using PyBoolNet
def bench_pyboolnet(models, timeout=120, number=5, max_output=1000):
    for model in models:
        print(model)        
        successes = 0
        total_time = 0
        nb_tspaces = 0
        for i in range(number):
            signal.alarm(timeout)
            try:
                start = time.perf_counter()
                primes = bnet2primes(model + ".bnet")
                tspaces = compute_trap_spaces(primes, "min", max_output=max_output)
                nb_tspaces = len(tspaces)
                end = time.perf_counter()
                total_time += end - start
                successes += 1
            except TimeOutException:
                total_time += timeout
            signal.alarm(0)
        if successes > 0:
            if successes < number:
                print(f"{number - successes} failures…")
            print(f"{nb_tspaces} trap spaces {total_time/number:.2f}\n===")
        else:
            print("DNF\n===")

In [3]:
# compute trapspaces using mpbn
def bench_mpbn(models, timeout=120, number=5, max_output=1000):
    for model in models:
        print(model)
        successes = 0
        total_time = 0
        nb_tspaces = 0
        for i in range(number):
            signal.alarm(timeout)
            try:
                start = time.perf_counter()
                mbn = mpbn.load(model + ".bnet")
                tspaces = list(mbn.attractors(limit=max_output))
                nb_tspaces = len(tspaces)
                end = time.perf_counter()
                total_time += end - start
                successes += 1
            except AssertionError:
                if i == 0:
                    print("Non-monotonic")
            except TimeOutException:
                total_time += timeout
            signal.alarm(0)
        if successes > 0:
            if successes < number:
                print(f"{number - successes} failures…")
            print(f"{nb_tspaces} trap spaces {total_time/number:.2f}\n===")
        else:
            print("DNF\n===")

In [4]:
# compute trapspaces using trappist
def bench_trappist(models, timeout=120, number=5, with_biolqm=True, max_output=1000, **kwargs):
    for model in models:
        print(model)
        successes = 0
        total_time = 0
        nb_tspaces = 0
        for i in range(number):
            signal.alarm(timeout)
            try:
                start = time.perf_counter()
                if with_biolqm:
                    biolqm.save(biolqm.load(model + ".bnet"), model + ".pnml")
                    tspaces = trappist.compute_trap_spaces(model + ".pnml", max_output=max_output, **kwargs)
                else:
                    tspaces = trappist.compute_trap_spaces(model + ".bnet", max_output=max_output, **kwargs)
                nb_tspaces = len(list(tspaces))
                end = time.perf_counter()
                total_time += end - start
                successes += 1
            except (TimeOutException, CalledProcessError, Py4JJavaError):
                total_time += timeout
            signal.alarm(0)
            time.sleep(1)
        if successes > 0:
            if successes < number:
                print(f"{number - successes} failures…")
            print(f"{nb_tspaces} trap spaces {total_time/number:.2f}\n===")
        else:
            print("DNF\n===")

## PyBoolNet repository

Unfortunately all models are very easy

In [5]:
# all models from the PyBoolNet repository https://github.com/hklarner/pyboolnet/tree/master/pyboolnet/repository
models = get_all_names()

# save .bnet files locally
for model in models:
    net = get_bnet(model)
    with open(model + ".bnet", "wt") as f:
        print(net, file=f)

In [6]:
# bench_pyboolnet(models)

arellano_rootstem
4 trap spaces 0.05
===
calzone_cellfate
27 trap spaces 0.03
===
dahlhaus_neuroplastoma
32 trap spaces 0.06
===
davidich_yeast
12 trap spaces 0.04
===
dinwoodie_life
7 trap spaces 0.03
===
dinwoodie_stomatal
1 trap spaces 0.02
===
faure_cellcycle
2 trap spaces 0.05
===
grieco_mapk
18 trap spaces 0.06
===
irons_yeast
1 trap spaces 0.06
===
jaoude_thdiff
1000 trap spaces 1.92
===
klamt_tcr
8 trap spaces 0.04
===
krumsiek_myeloid
6 trap spaces 0.03
===
multivalued
4 trap spaces 0.03
===
n12c5
5 trap spaces 38.80
===
n3s1c1a
2 trap spaces 0.02
===
n3s1c1b
2 trap spaces 0.02
===
n5s3
3 trap spaces 0.03
===
n6s1c2
3 trap spaces 0.04
===
n7s3
3 trap spaces 0.02
===
raf
2 trap spaces 0.02
===
randomnet_n15k3
3 trap spaces 0.04
===
randomnet_n7k3
10 trap spaces 0.03
===
remy_tumorigenesis
25 trap spaces 2.05
===
saadatpour_guardcell
1 trap spaces 0.02
===
selvaggio_emt
1000 trap spaces 1.09
===
tournier_apoptosis
3 trap spaces 0.03
===
xiao_wnt5a
4 trap spaces 0.02
===
zhang_tl

In [7]:
# bench_mpbn(models)

arellano_rootstem
4 trap spaces 0.01
===
calzone_cellfate
27 trap spaces 0.02
===
dahlhaus_neuroplastoma
32 trap spaces 0.02
===
davidich_yeast
12 trap spaces 0.02
===
dinwoodie_life
7 trap spaces 0.01
===
dinwoodie_stomatal
1 trap spaces 0.00
===
faure_cellcycle
2 trap spaces 0.02
===
grieco_mapk
18 trap spaces 0.04
===
irons_yeast
1 trap spaces 0.02
===
jaoude_thdiff
1000 trap spaces 1.32
===
klamt_tcr
8 trap spaces 0.01
===
krumsiek_myeloid
6 trap spaces 0.01
===
multivalued
4 trap spaces 0.00
===
n12c5
5 trap spaces 0.02
===
n3s1c1a
2 trap spaces 0.00
===
n3s1c1b
2 trap spaces 0.00
===
n5s3
Non-monotonic
DNF
===
n6s1c2
3 trap spaces 0.01
===
n7s3
3 trap spaces 0.00
===
raf
2 trap spaces 0.00
===
randomnet_n15k3
Non-monotonic
DNF
===
randomnet_n7k3
Non-monotonic
DNF
===
remy_tumorigenesis
25 trap spaces 0.04
===
saadatpour_guardcell
1 trap spaces 0.00
===
selvaggio_emt
1000 trap spaces 0.76
===
tournier_apoptosis
3 trap spaces 0.01
===
xiao_wnt5a
4 trap spaces 0.00
===
zhang_tlgl
15

In [8]:
# bench_trappist(models)

arellano_rootstem
4 trap spaces 0.08
===
calzone_cellfate
27 trap spaces 0.04
===
dahlhaus_neuroplastoma
32 trap spaces 0.04
===
davidich_yeast
12 trap spaces 0.03
===
dinwoodie_life
7 trap spaces 0.03
===
dinwoodie_stomatal
1 trap spaces 0.03
===
faure_cellcycle
2 trap spaces 0.03
===
grieco_mapk
18 trap spaces 0.07
===
irons_yeast
1 trap spaces 0.05
===
jaoude_thdiff
1000 trap spaces 0.22
===
klamt_tcr
8 trap spaces 0.05
===
krumsiek_myeloid
6 trap spaces 0.02
===
multivalued
4 trap spaces 0.02
===
n12c5
5 trap spaces 0.04
===
n3s1c1a
2 trap spaces 0.01
===
n3s1c1b
2 trap spaces 0.01
===
n5s3
3 trap spaces 0.02
===
n6s1c2
3 trap spaces 0.02
===
n7s3
3 trap spaces 0.02
===
raf
2 trap spaces 0.02
===
randomnet_n15k3
3 trap spaces 0.04
===
randomnet_n7k3
10 trap spaces 0.02
===
remy_tumorigenesis
25 trap spaces 0.07
===
saadatpour_guardcell
1 trap spaces 0.02
===
selvaggio_emt
1000 trap spaces 0.20
===
tournier_apoptosis
3 trap spaces 0.04
===
xiao_wnt5a
4 trap spaces 0.02
===
zhang_tlg

In [9]:
# bench_trappist(models, with_biolqm=False)

arellano_rootstem
4 trap spaces 0.02
===
calzone_cellfate
27 trap spaces 0.03
===
dahlhaus_neuroplastoma
32 trap spaces 0.03
===
davidich_yeast
12 trap spaces 0.03
===
dinwoodie_life
7 trap spaces 0.03
===
dinwoodie_stomatal
1 trap spaces 0.02
===
faure_cellcycle
2 trap spaces 0.03
===
grieco_mapk
18 trap spaces 0.05
===
irons_yeast
1 trap spaces 0.03
===
jaoude_thdiff
1000 trap spaces 0.20
===
klamt_tcr
8 trap spaces 0.04
===
krumsiek_myeloid
6 trap spaces 0.02
===
multivalued
4 trap spaces 0.02
===
n12c5
5 trap spaces 0.04
===
n3s1c1a
2 trap spaces 0.02
===
n3s1c1b
2 trap spaces 0.01
===
n5s3
3 trap spaces 0.02
===
n6s1c2
3 trap spaces 0.02
===
n7s3
3 trap spaces 0.02
===
raf
2 trap spaces 0.02
===
randomnet_n15k3
3 trap spaces 0.04
===
randomnet_n7k3
10 trap spaces 0.02
===
remy_tumorigenesis
25 trap spaces 0.06
===
saadatpour_guardcell
1 trap spaces 0.02
===
selvaggio_emt
1000 trap spaces 0.18
===
tournier_apoptosis
3 trap spaces 0.02
===
xiao_wnt5a
4 trap spaces 0.02
===
zhang_tlg

In [10]:
# bench_trappist(models, method='sat')

arellano_rootstem
4 trap spaces 0.01
===
calzone_cellfate
27 trap spaces 0.02
===
dahlhaus_neuroplastoma
32 trap spaces 0.03
===
davidich_yeast
12 trap spaces 0.01
===
dinwoodie_life
7 trap spaces 0.01
===
dinwoodie_stomatal
1 trap spaces 0.01
===
faure_cellcycle
2 trap spaces 0.01
===
grieco_mapk
18 trap spaces 0.06
===
irons_yeast
1 trap spaces 0.01
===
jaoude_thdiff
1000 trap spaces 0.73
===
klamt_tcr
8 trap spaces 0.02
===
krumsiek_myeloid
6 trap spaces 0.01
===
multivalued
4 trap spaces 0.00
===
n12c5
5 trap spaces 0.03
===
n3s1c1a
2 trap spaces 0.00
===
n3s1c1b
2 trap spaces 0.00
===
n5s3
3 trap spaces 0.01
===
n6s1c2
3 trap spaces 0.01
===
n7s3
3 trap spaces 0.00
===
raf
2 trap spaces 0.00
===
randomnet_n15k3
3 trap spaces 0.02
===
randomnet_n7k3
10 trap spaces 0.01
===
remy_tumorigenesis
25 trap spaces 0.04
===
saadatpour_guardcell
1 trap spaces 0.01
===
selvaggio_emt
1000 trap spaces 0.39
===
tournier_apoptosis
3 trap spaces 0.01
===
xiao_wnt5a
4 trap spaces 0.01
===
zhang_tlg

In [11]:
bench_trappist(models, with_biolqm=False, method='naive')

arellano_rootstem
4 trap spaces 0.02
===
calzone_cellfate
27 trap spaces 0.03
===
dahlhaus_neuroplastoma
32 trap spaces 0.03
===
davidich_yeast
12 trap spaces 0.02
===
dinwoodie_life
7 trap spaces 0.02
===
dinwoodie_stomatal
1 trap spaces 0.01
===
faure_cellcycle
2 trap spaces 0.02
===
grieco_mapk
18 trap spaces 0.05
===
irons_yeast
1 trap spaces 0.03
===
jaoude_thdiff
1000 trap spaces 0.14
===
klamt_tcr
8 trap spaces 0.03
===
krumsiek_myeloid
6 trap spaces 0.02
===
multivalued
4 trap spaces 0.02
===
n12c5
5 trap spaces 0.03
===
n3s1c1a
2 trap spaces 0.01
===
n3s1c1b
2 trap spaces 0.01
===
n5s3
3 trap spaces 0.02
===
n6s1c2
3 trap spaces 0.02
===
n7s3
3 trap spaces 0.01
===
raf
2 trap spaces 0.01
===
randomnet_n15k3
3 trap spaces 0.03
===
randomnet_n7k3
10 trap spaces 0.02
===
remy_tumorigenesis
25 trap spaces 0.04
===
saadatpour_guardcell
1 trap spaces 0.02
===
selvaggio_emt
1000 trap spaces 0.11
===
tournier_apoptosis
3 trap spaces 0.02
===
xiao_wnt5a
4 trap spaces 0.01
===
zhang_tlg

## Local set of big models

Collected from various bibliographic sources (see article for details)

In [12]:
# all models from the trappist's benchmark directory, sorted by size (increasing)
models = [(str(Path().absolute() / "models" / file.name[:-5]), file.stat().st_size) for file in os.scandir("models") if file.name.endswith(".bnet")]
models.sort(key=lambda model: model[1])
models = map(lambda model: model[0], models)

In [13]:
# bench_pyboolnet(models)

/notebook/models/Executable_file_for_CaSQ_derived_mast_cell_activation_model_BCC
1000 trap spaces 0.16
===
/notebook/models/Colitis_associated_colon_cancer
10 trap spaces 0.07
===
/notebook/models/IL_6_Signalling
1000 trap spaces 0.17
===
/notebook/models/SIGNALING-PATHWAY-FOR-BUTANOL-PRODUCTION
1000 trap spaces 0.20
===
/notebook/models/TLGLSurvival
318 trap spaces 0.23
===
/notebook/models/INTERFERON-1
1000 trap spaces 2.82
===
/notebook/models/Influenza_A_Virus_Replication_Cycle
1000 trap spaces 8.75
===
/notebook/models/InflammatoryBowelDisease
Timeout
Timeout
Timeout
Timeout
Timeout
DNF
===
/notebook/models/HMOX1-PATHWAY
1000 trap spaces 1.65
===
/notebook/models/VIRUS-REPLICATION-CYCLE
Timeout
Timeout
Timeout
Timeout
Timeout
DNF
===
/notebook/models/Corral_ThIL17diff_15jan2021
Timeout
Timeout
Timeout
Timeout
Timeout
DNF
===
/notebook/models/korkut_2015a
Timeout
Timeout
Timeout
Timeout
Timeout
DNF
===
/notebook/models/TCR-TLR5-SIGNALING-2018
48 trap spaces 0.54
===
/notebook/model

In [14]:
# bench_mpbn(models)

/notebook/models/Executable_file_for_CaSQ_derived_mast_cell_activation_model_BCC
1000 trap spaces 0.89
===
/notebook/models/Colitis_associated_colon_cancer
10 trap spaces 0.04
===
/notebook/models/IL_6_Signalling
1000 trap spaces 1.01
===
/notebook/models/SIGNALING-PATHWAY-FOR-BUTANOL-PRODUCTION
1000 trap spaces 0.88
===
/notebook/models/TLGLSurvival
318 trap spaces 0.30
===
/notebook/models/INTERFERON-1
1000 trap spaces 1.42
===
/notebook/models/Influenza_A_Virus_Replication_Cycle
1000 trap spaces 1.54
===
/notebook/models/InflammatoryBowelDisease
Non-monotonic
DNF
===
/notebook/models/HMOX1-PATHWAY
1000 trap spaces 1.98
===
/notebook/models/VIRUS-REPLICATION-CYCLE
1000 trap spaces 2.21
===
/notebook/models/Corral_ThIL17diff_15jan2021
1000 trap spaces 1.18
===
/notebook/models/korkut_2015a
1000 trap spaces 1.36
===
/notebook/models/TCR-TLR5-SIGNALING-2018
48 trap spaces 0.18
===
/notebook/models/RA_apoptosis_executable_module_BCC
1000 trap spaces 2.27
===
/notebook/models/Regan2020_Ad

In [15]:
# bench_trappist(models)

/notebook/models/Executable_file_for_CaSQ_derived_mast_cell_activation_model_BCC
1000 trap spaces 0.12
===
/notebook/models/Colitis_associated_colon_cancer
10 trap spaces 0.08
===
/notebook/models/IL_6_Signalling
1000 trap spaces 0.15
===
/notebook/models/SIGNALING-PATHWAY-FOR-BUTANOL-PRODUCTION
1000 trap spaces 0.13
===
/notebook/models/TLGLSurvival
318 trap spaces 0.12
===
/notebook/models/INTERFERON-1
1000 trap spaces 0.19
===
/notebook/models/Influenza_A_Virus_Replication_Cycle
1000 trap spaces 0.26
===
/notebook/models/InflammatoryBowelDisease
1 trap spaces 18.06
===
/notebook/models/HMOX1-PATHWAY
1000 trap spaces 0.21
===
/notebook/models/VIRUS-REPLICATION-CYCLE
1000 trap spaces 0.25
===
/notebook/models/Corral_ThIL17diff_15jan2021
1000 trap spaces 0.19
===
/notebook/models/korkut_2015a
1000 trap spaces 0.52
===
/notebook/models/TCR-TLR5-SIGNALING-2018
48 trap spaces 0.14
===
/notebook/models/RA_apoptosis_executable_module_BCC
1000 trap spaces 0.26
===
/notebook/models/Regan2020_

In [16]:
# bench_trappist(models, max_output=0)

/notebook/models/Executable_file_for_CaSQ_derived_mast_cell_activation_model_BCC
Timeout
Timeout
Timeout
Timeout
Timeout
DNF
===
/notebook/models/Colitis_associated_colon_cancer
10 trap spaces 0.09
===
/notebook/models/IL_6_Signalling
32768 trap spaces 2.69
===
/notebook/models/SIGNALING-PATHWAY-FOR-BUTANOL-PRODUCTION
8192 trap spaces 0.57
===
/notebook/models/TLGLSurvival
318 trap spaces 0.11
===
/notebook/models/INTERFERON-1
Timeout
Timeout
Timeout
Timeout
Timeout
DNF
===
/notebook/models/Influenza_A_Virus_Replication_Cycle
10128 trap spaces 1.39
===
/notebook/models/InflammatoryBowelDisease
1 trap spaces 17.92
===
/notebook/models/HMOX1-PATHWAY
Timeout
Timeout
Timeout
Timeout
Timeout
DNF
===
/notebook/models/VIRUS-REPLICATION-CYCLE
Timeout
Timeout
Timeout
Timeout
Timeout
DNF
===
/notebook/models/Corral_ThIL17diff_15jan2021
Timeout
Timeout
Timeout
Timeout
Timeout
DNF
===
/notebook/models/korkut_2015a
18556 trap spaces 3.54
===
/notebook/models/TCR-TLR5-SIGNALING-2018
48 trap spaces 0

In [17]:
# bench_trappist(models, with_biolqm=False)

/notebook/models/Executable_file_for_CaSQ_derived_mast_cell_activation_model_BCC
1000 trap spaces 0.13
===
/notebook/models/Colitis_associated_colon_cancer
10 trap spaces 0.06
===
/notebook/models/IL_6_Signalling
1000 trap spaces 0.13
===
/notebook/models/SIGNALING-PATHWAY-FOR-BUTANOL-PRODUCTION
1000 trap spaces 0.12
===
/notebook/models/TLGLSurvival
318 trap spaces 0.10
===
/notebook/models/INTERFERON-1
1000 trap spaces 0.17
===
/notebook/models/Influenza_A_Virus_Replication_Cycle
1000 trap spaces 0.20
===
/notebook/models/InflammatoryBowelDisease
1 trap spaces 1.71
===
/notebook/models/HMOX1-PATHWAY
1000 trap spaces 0.20
===
/notebook/models/VIRUS-REPLICATION-CYCLE
1000 trap spaces 0.23
===
/notebook/models/Corral_ThIL17diff_15jan2021
1000 trap spaces 0.18
===
/notebook/models/korkut_2015a
1000 trap spaces 0.39
===
/notebook/models/TCR-TLR5-SIGNALING-2018
48 trap spaces 0.13
===
/notebook/models/RA_apoptosis_executable_module_BCC
1000 trap spaces 0.25
===
/notebook/models/Regan2020_A

In [18]:
# bench_trappist(models, method='sat')

/notebook/models/Executable_file_for_CaSQ_derived_mast_cell_activation_model_BCC
1000 trap spaces 0.47
===
/notebook/models/Colitis_associated_colon_cancer
10 trap spaces 0.07
===
/notebook/models/IL_6_Signalling
1000 trap spaces 0.51
===
/notebook/models/SIGNALING-PATHWAY-FOR-BUTANOL-PRODUCTION
1000 trap spaces 0.41
===
/notebook/models/TLGLSurvival
318 trap spaces 0.17
===
/notebook/models/INTERFERON-1
1000 trap spaces 0.79
===
/notebook/models/Influenza_A_Virus_Replication_Cycle
1000 trap spaces 0.99
===
/notebook/models/InflammatoryBowelDisease
1 trap spaces 3.17
===
/notebook/models/HMOX1-PATHWAY
1000 trap spaces 1.04
===
/notebook/models/VIRUS-REPLICATION-CYCLE
1000 trap spaces 1.13
===
/notebook/models/Corral_ThIL17diff_15jan2021
1000 trap spaces 0.57
===
/notebook/models/korkut_2015a
1000 trap spaces 0.89
===
/notebook/models/TCR-TLR5-SIGNALING-2018
48 trap spaces 0.18
===
/notebook/models/RA_apoptosis_executable_module_BCC
1000 trap spaces 1.57
===
/notebook/models/Regan2020_A

In [19]:
bench_trappist(models, with_biolqm=False, method='naive')

/notebook/models/Executable_file_for_CaSQ_derived_mast_cell_activation_model_BCC
1000 trap spaces 0.10
===
/notebook/models/Colitis_associated_colon_cancer
10 trap spaces 0.05
===
/notebook/models/IL_6_Signalling
1000 trap spaces 0.11
===
/notebook/models/SIGNALING-PATHWAY-FOR-BUTANOL-PRODUCTION
1000 trap spaces 0.12
===
/notebook/models/TLGLSurvival
318 trap spaces 0.08
===
/notebook/models/INTERFERON-1
1000 trap spaces 0.15
===
/notebook/models/Influenza_A_Virus_Replication_Cycle
1000 trap spaces 0.16
===
/notebook/models/InflammatoryBowelDisease
1 trap spaces 0.09
===
/notebook/models/HMOX1-PATHWAY
1000 trap spaces 0.18
===
/notebook/models/VIRUS-REPLICATION-CYCLE
1000 trap spaces 0.19
===
/notebook/models/Corral_ThIL17diff_15jan2021
1000 trap spaces 0.15
===
/notebook/models/korkut_2015a
1000 trap spaces 0.17
===
/notebook/models/TCR-TLR5-SIGNALING-2018
48 trap spaces 0.10
===
/notebook/models/RA_apoptosis_executable_module_BCC
1000 trap spaces 0.23
===
/notebook/models/Regan2020_A

# Hard Models

In [20]:
# models from the hard_models directory, sorted by size (increasing)
models = [(str(Path().absolute() / "hard_models" / file.name[:-5]), file.stat().st_size) for file in os.scandir("hard_models") if file.name.endswith(".bnet")]
models.sort(key=lambda model: model[1])
models = map(lambda model: model[0], models)

In [21]:
bench_trappist(models, with_biolqm=False, method='naive')